<a href="https://colab.research.google.com/github/TazeemKhan9/Hate-Speech-Detector/blob/main/Modelling/Hate_Speech_Project_(LSTM).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Importing all the libraries**

In [ ]:
import nltk
nltk.download()

NLTK Downloader
---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> d

Download which package (l=list; x=cancel)?
  Identifier> all
       | 
       | Downloading package abc to /root/nltk_data...
       |   Unzipping corpora/abc.zip.
       | Downloading package alpino to /root/nltk_data...
       |   Unzipping corpora/alpino.zip.
       | Downloading package biocreative_ppi to /root/nltk_data...
       |   Unzipping corpora/biocreative_ppi.zip.
       | Downloading package brown to /root/nltk_data...
       |   Unzipping corpora/brown.zip.
       | Downloading package brown_tei to /root/nltk_data...
       |   Unzipping corpora/brown_tei.zip.
       | Downloading package cess_cat to /root/nltk_data...
       |   Unzipping corpora/cess_cat.zip.
       | Downloading package cess_esp to /root/nltk_data...
 

True

In [ ]:
!pip install textstat

     |████████████████████████████████| 102kB 9.2MB/s 
     |████████████████████████████████| 1.9MB 31.8MB/s 


In [ ]:
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import *
import string
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import confusion_matrix
import seaborn
from textstat.textstat import *
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
import numpy as np
from nltk.sentiment.vader import SentimentIntensityAnalyzer as VS
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
%matplotlib inline

/usr/local/lib/python3.7/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


# **Reading the Dataset**

In [ ]:
dataset = pd.read_csv("HateSpeechData.csv")
dataset

,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet
0,0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...
...,...,...,...,...,...,...,...
24778,25291,3,0,2,1,1,you's a muthaf***in lie &#8220;@LifeAsKing: @2...
24779,25292,3,0,1,2,2,"you've gone and broke the wrong heart baby, an..."
24780,25294,3,0,3,0,1,young buck wanna eat!!.. dat nigguh like I ain...
24781,25295,6,0,6,0,1,youu got wild bitches tellin you lies


In [ ]:
dataset['text length'] = dataset['tweet'].apply(len)
print(dataset.head())

   Unnamed: 0  ...  text length
0           0  ...          140
1           1  ...           85
2           2  ...          120
3           3  ...           62
4           4  ...          137

[5 rows x 8 columns]


In [ ]:
tweet=dataset.tweet

# **Preprocessing**

In [ ]:
## 1. Removal of punctuation and capitlization
## 2. Tokenizing
## 3. Removal of stopwords
## 4. Stemming

stopwords = nltk.corpus.stopwords.words("english")

#extending the stopwords to include other words used in twitter such as retweet(rt) etc.
other_exclusions = ["#ff", "ff", "rt"]
stopwords.extend(other_exclusions)
stemmer = PorterStemmer()

def preprocess(tweet):  
    
    # removal of extra spaces
    regex_pat = re.compile(r'\s+')
    tweet_space = tweet.str.replace(regex_pat, ' ')

    # removal of @name[mention]
    regex_pat = re.compile(r'@[\w\-]+')
    tweet_name = tweet_space.str.replace(regex_pat, '')

    # removal of links[https://abc.com]
    giant_url_regex =  re.compile('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|'
            '[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
    tweets = tweet_name.str.replace(giant_url_regex, '')
    
    # removal of punctuations and numbers
    punc_remove = tweets.str.replace("[^a-zA-Z]", " ")
    # remove whitespace with a single space
    newtweet=punc_remove.str.replace(r'\s+', ' ')
    # remove leading and trailing whitespace
    newtweet=newtweet.str.replace(r'^\s+|\s+?$','')
    # replace normal numbers with numbr
    newtweet=newtweet.str.replace(r'\d+(\.\d+)?','numbr')
    # removal of capitalization
    tweet_lower = newtweet.str.lower()
    
    # tokenizing
    tokenized_tweet = tweet_lower.apply(lambda x: x.split())
    
    # removal of stopwords
    tokenized_tweet=  tokenized_tweet.apply(lambda x: [item for item in x if item not in stopwords])
    
    # stemming of the tweets
    tokenized_tweet = tokenized_tweet.apply(lambda x: [stemmer.stem(i) for i in x]) 
    
    for i in range(len(tokenized_tweet)):
        tokenized_tweet[i] = ' '.join(tokenized_tweet[i])
        tweets_p= tokenized_tweet
    
    return tweets_p

processed_tweets = preprocess(tweet)   

dataset['processed_tweets'] = processed_tweets
print(dataset[["tweet","processed_tweets"]].head(10))

                                               tweet                                   processed_tweets
0  !!! RT @mayasolovely: As a woman you shouldn't...  woman complain clean hous amp man alway take t...
1  !!!!! RT @mleew17: boy dats cold...tyga dwn ba...  boy dat cold tyga dwn bad cuffin dat hoe st place
2  !!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...         dawg ever fuck bitch start cri confus shit
3  !!!!!!!!! RT @C_G_Anderson: @viva_based she lo...                                   look like tranni
4  !!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...     shit hear might true might faker bitch told ya
5  !!!!!!!!!!!!!!!!!!"@T_Madison_x: The shit just...      shit blow claim faith somebodi still fuck hoe
6  !!!!!!"@__BrighterDays: I can not just sit up ...              sit hate anoth bitch got much shit go
7  !!!!&#8220;@selfiequeenbri: cause I'm tired of...            caus tire big bitch come us skinni girl
8  " &amp; you might not get ya bitch back &amp; ...            

# LSTM MODEL (VER 1)

In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras import regularizers
from keras.utils import to_categorical
max_words = 5000
max_len = 200

tokenizer = Tokenizer(num_words=max_words)
vocabSize = len(tokenizer.word_index) + 1
tokenizer.fit_on_texts(dataset['processed_tweets'])
X = tokenizer.texts_to_sequences(dataset['processed_tweets'])
X = pad_sequences(X)
Y = dataset['class']
Y = to_categorical(Y, 3)
print(X)

[[   0    0    0 ...   88   52   12]
 [   0    0    0 ...    2  448  386]
 [   0    0    0 ...  233  771    9]
 ...
 [   0    0    0 ...  103  102  249]
 [   0    0    0 ...    1 1707  198]
 [   0    0    0 ...   99   14   53]]


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X ,Y, test_size=0.2, random_state=5)

In [ ]:
from keras.models import Sequential
from keras import layers
from keras import regularizers
from keras import backend as K
from keras.callbacks import ModelCheckpoint
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import Embedding
model = Sequential()
model.add(Embedding(max_words,30))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.5))
model.add(Dense(3, activation='softmax'))

In [ ]:
model.compile(optimizer='adam',loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
checkpoint1 = ModelCheckpoint("best_model1.hdf5", monitor='val_accuracy', verbose=1,save_best_only=True, mode='auto', period=1,save_weights_only=False)
history = model.fit(X_train, Y_train, epochs=10,validation_data=(X_test, Y_test),callbacks=[checkpoint1])

Epoch 1/10
620/620 [==============================] - 58s 86ms/step - loss: 0.5509 - accuracy: 0.8098 - val_loss: 0.3049 - val_accuracy: 0.8999

Epoch 00001: val_accuracy improved from -inf to 0.89994, saving model to best_model1.hdf5
Epoch 2/10
620/620 [==============================] - 54s 87ms/step - loss: 0.2468 - accuracy: 0.9155 - val_loss: 0.2709 - val_accuracy: 0.9050

Epoch 00002: val_accuracy improved from 0.89994 to 0.90498, saving model to best_model1.hdf5
Epoch 3/10
620/620 [==============================] - 54s 87ms/step - loss: 0.2002 - accuracy: 0.9303 - val_loss: 0.2849 - val_accuracy: 0.9054

Epoch 00003: val_accuracy improved from 0.90498 to 0.90539, saving model to best_model1.hdf5
Epoch 4/10
620/620 [==============================] - 54s 87ms/step - loss: 0.1666 - accuracy: 0.9406 - val_loss: 0.3159 - val_accuracy: 0.9030

Epoch 00004: val_accuracy did not improve from 0.90539
Epoch 5/10
620/620 [==============================] - 53s 86ms/step - loss: 0.1442 - accu

In [ ]:
sentiment = ['Hate Speech','Offensive Language','No Issues']

In [ ]:
input='You are a good man'
input = tokenizer.texts_to_sequences([input])

In [ ]:
test = pad_sequences(input, maxlen=max_len)
sentiment[np.around(model.predict(test), decimals=0).argmax(axis=1)[0]]

'Offensive Language'

In [ ]:
model.predict(input)

array([[0.00496075, 0.94765407, 0.04738529]], dtype=float32)

# Preprocessing (VER 2)

In [ ]:
!pip install swachhdata -q

     |████████████████████████████████| 133kB 16.0MB/s 
     |████████████████████████████████| 102kB 8.5MB/s 
     |████████████████████████████████| 327kB 16.3MB/s 
     |████████████████████████████████| 266kB 37.0MB/s 


In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("https://github.com/TazeemKhan9/Hate-Speech-Detector/blob/main/Data/HateSpeechData.csv?raw=true")
df

,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet
0,0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...
...,...,...,...,...,...,...,...
24778,25291,3,0,2,1,1,you's a muthaf***in lie &#8220;@LifeAsKing: @2...
24779,25292,3,0,1,2,2,"you've gone and broke the wrong heart baby, an..."
24780,25294,3,0,3,0,1,young buck wanna eat!!.. dat nigguh like I ain...
24781,25295,6,0,6,0,1,youu got wild bitches tellin you lies


In [ ]:
df['tweet'] = df['tweet'].str.lower()

In [ ]:
from swachhdata.text import *

In [ ]:
url = urlRecast(process='remove', verbose=1)
df['tweet'] = url.setup_recast(df['tweet'])

In [ ]:
html = htmlRecast(verbose=1)
df['tweet'] = html.setup_recast(df['tweet'])

In [ ]:
esr = EscapeSequenceRecast(verbose=1)
df['tweet'] = esr.setup_recast(df['tweet'])

In [ ]:
mention = MentionRecast(process='remove', verbose=1)
df['tweet'] = mention.setup_recast(df['tweet'])

In [ ]:
contraction = ContractionsRecast(verbose=1)
df['tweet'] = contraction.setup_recast(df['tweet'])

In [ ]:
emoji = EmojiRecast(process='remove', verbose=1)
df['tweet'] = emoji.setup_recast(df['tweet'])

In [ ]:
hashtag = HashtagRecast(process='remove', verbose=1)
df['tweet'] = hashtag.setup_recast(df['tweet'])

In [ ]:
stopwords = StopWordsRecast(package='nltk', verbose=1)
df['tweet'] = stopwords.setup_recast(df['tweet'])

In [ ]:
num = NumberRecast(process='remove', verbose=1)
df['tweet'] = num.setup_recast(df['tweet'])

In [ ]:
alp = AlphabetRecast(verbose=1)
df['tweet'] = alp.setup_recast(df['tweet'])

In [ ]:
punc = PunctuationRecast(verbose=1)
df['tweet'] = punc.setup_recast(df['tweet'])

In [ ]:
lem = LemmatizationRecast(verbose=1)
df['tweet'] = lem.setup_recast(df['tweet'])

In [ ]:
df['tweet']

0        rt woman complain cleaning house man always ta...
1        rt boy dat coldtyga dwn bad cuffin dat hoe st ...
2        rt dawg rt ever fuck bitch start cry confuse shit
3                                      rt look like tranny
4        rt shit hear might true might faker bitch told ya
                               ...                        
24778    you s muthaf in lie right tl trash now mine bi...
24779        go broke wrong heart baby drove redneck crazy
24780       young buck want eat dat nigguh like fuckin dis
24781                       youu get wild bitch tellin lie
24782    ruffle ntac eileen dahlia beautiful color comb...
Name: tweet, Length: 24783, dtype: object

In [ ]:
df['tweet'].to_csv("cleaned_tweet.csv")

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from string import punctuation #Remove puntuationpunctuation 
from nltk.corpus import stopwords #Remove stop words
from nltk.tokenize import word_tokenize 
from nltk.stem import LancasterStemmer
from nltk.tag import pos_tag

# LSTM MODEL (VER 2)

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
tokenizer =Tokenizer()
tokenizer.fit_on_texts(df['tweet'].values)
vocabSize = len(tokenizer.word_index) + 1
X = tokenizer.texts_to_sequences(df['tweet'])
X = pad_sequences(X)
Y=  df['class']
Y = to_categorical(Y, 3)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.2, random_state=5)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import Embedding
from keras.callbacks import ModelCheckpoint

model = Sequential()
model.add(Embedding(vocabSize, 128, input_length = 30))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(3, activation='softmax'))

In [ ]:
model.compile(optimizer='adam',loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
checkpoint1 = ModelCheckpoint("best_model2.hdf5", monitor='val_accuracy', verbose=1,save_best_only=True, mode='auto', period=1,save_weights_only=False)
history = model.fit(X_train, Y_train, epochs=10,validation_data=(X_test, Y_test),callbacks=[checkpoint1])

Epoch 1/10
620/620 [==============================] - 120s 185ms/step - loss: 0.4841 - accuracy: 0.8289 - val_loss: 0.2817 - val_accuracy: 0.8961

Epoch 00001: val_accuracy improved from -inf to 0.89611, saving model to best_model2.hdf5
Epoch 2/10
620/620 [==============================] - 113s 183ms/step - loss: 0.2151 - accuracy: 0.9234 - val_loss: 0.3068 - val_accuracy: 0.8929

Epoch 00002: val_accuracy did not improve from 0.89611
Epoch 3/10
620/620 [==============================] - 114s 184ms/step - loss: 0.1373 - accuracy: 0.9522 - val_loss: 0.3460 - val_accuracy: 0.8862

Epoch 00003: val_accuracy did not improve from 0.89611
Epoch 4/10
620/620 [==============================] - 114s 183ms/step - loss: 0.0961 - accuracy: 0.9668 - val_loss: 0.3802 - val_accuracy: 0.8866

Epoch 00004: val_accuracy did not improve from 0.89611
Epoch 5/10
620/620 [==============================] - 114s 184ms/step - loss: 0.0815 - accuracy: 0.9724 - val_loss: 0.4624 - val_accuracy: 0.8721

Epoch 0000

In [ ]:
def process(tweet):
    return TextRecast(tweet, urlRecast = {'process': 'remove'},
                      htmlRecast = True,
                      EscapeSequenceRecast = True,
                      MentionRecast = {'process': 'extract_remove'},
                      ContractionsRecast = True,
                      CaseRecast = {'process': 'lower'},
                      EmojiRecast = {'process': 'remove', 'space_out': False},
                      HashtagRecast = {'process': 'remove'},
                      StopWordsRecast = {'package': 'nltk', 'stopwords': None},
                      NumberRecast = {'process': 'remove', 'seperator': None},
                      PunctuationRecast = True,
                      LemmatizationRecast = {'package':'nltk'})

In [ ]:
df['New_tweet']= process(df['tweet'])

In [ ]:
from keras.models import load_model
model2 = load_model('best_model2.hdf5')

In [ ]:
tweet="Kritik gay"
tweet = process(tweet)

In [ ]:
input = tokenizer.texts_to_sequences([tweet])

In [ ]:
input

[[182]]

In [ ]:
test = pad_sequences(input, maxlen=30)
sentiment[np.around(model.predict(test), decimals=0).argmax(axis=1)[0]]

'Offensive Language'

In [ ]:
sentiment[np.around(model2.predict(test), decimals=0).argmax(axis=1)[0]]

'Hate Speech'

In [ ]:
model.predict(test)

array([[0.46179825, 0.5006757 , 0.03752605]], dtype=float32)

In [ ]:
model2.predict(test)

array([[0.31703746, 0.3370897 , 0.34587285]], dtype=float32)